In [1]:
import string
import math
import random

In [2]:
def clear_message(text, stop_symbols):
    text = text.lower()
    return ''.join([i for i in text if i not in stop_symbols])


def create_cipher_dict(cipher):
    cipher_dict = {}
    for i in range(len(alphabet)):
        cipher_dict[alphabet[i]] = cipher[i]
    return cipher_dict


def reverse_cipher(cipher):
    cipher_dict = create_cipher_dict(cipher)
    reverse_dict = {value: key for key, value in cipher_dict.items()}
    reversed_cipher = ''
    for i in alphabet:
        reversed_cipher += reverse_dict[i]
    return reversed_cipher


def create_ciphertext(text, cipher):
    cipher_dict = create_cipher_dict(cipher)
    text = list(text)
    ciphertext = ''
    for elem in text:
        if elem in cipher_dict:
            ciphertext += cipher_dict[elem]
        else:
            ciphertext += ' '
    return ciphertext


def compute_frequency_in_corpus(longtext_path):
    duplets_frequency = {}
    try:
        with open(longtext_path) as f:
            for line in f:
                data = list(line.strip())
                for i in range(len(data)-1):
                    alpha_i = data[i].lower()
                    alpha_j = data[i+1].lower()
                    if alpha_i in alphabet and alpha_j in alphabet:
                        key = alpha_i+alpha_j
                        if key in duplets_frequency:
                            duplets_frequency[key] += 1
                        else:
                            duplets_frequency[key] = 1
    except:
        with open(longtext_path, encoding='utf-8') as f:
            for line in f:
                data = list(line.strip())
                for i in range(len(data)-1):
                    alpha_i = data[i].lower()
                    alpha_j = data[i+1].lower()
                    if alpha_i in alphabet and alpha_j in alphabet:
                        key = alpha_i+alpha_j
                        if key in duplets_frequency:
                            duplets_frequency[key] += 1
                        else:
                            duplets_frequency[key] = 1
    return duplets_frequency


def compute_frequency_in_ciphertext(text):
    duplets_frequency = {}
    data = list(text.strip())
    for i in range(len(data)-1):
        alpha_i = data[i]
        alpha_j = data[i+1]
        if alpha_i in alphabet and alpha_j in alphabet:
            key = alpha_i+alpha_j
        key = alpha_i+alpha_j
        if key in duplets_frequency:
            duplets_frequency[key] += 1
        else:
            duplets_frequency[key] = 1
    return duplets_frequency


def compute_score(text, cipher, duplets_frequency_in_corpus):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = create_ciphertext(text, cipher)
    duplets_frequency_in_ciphertext = compute_frequency_in_ciphertext(decrypted_text)
    score = 0
    for duplet, count in duplets_frequency_in_ciphertext.items():
        if duplet in duplets_frequency_in_corpus:
            score += count*math.log(duplets_frequency_in_corpus[duplet])
    return score


def generate_cipher(cipher):
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:
        cipher = list(cipher)
        cipher[pos1], cipher[pos2] = cipher[pos2], cipher[pos1]
        return ''.join(cipher)


def random_coin(p):
    unif = random.uniform(0, 1)
    if unif >= p:
        return False
    else:
        return True


def get_random_key():
    str_var = list(alphabet)
    random.shuffle(str_var)
    random_key = ''.join(str_var)
    return random_key


def MCMC_decrypt(n_iter, cipher_text, duplets_frequency_in_corpus):
    current_cipher = alphabet
    best_state = ''
    score = 0
    for i in range(n_iter):
        proposed_cipher = generate_cipher(current_cipher)
        score_current_cipher = compute_score(
            cipher_text, current_cipher, duplets_frequency_in_corpus)
        score_proposed_cipher = compute_score(
            cipher_text, proposed_cipher, duplets_frequency_in_corpus)
        acceptance_probability = min(1, math.exp(
            score_proposed_cipher-score_current_cipher))
        if score_current_cipher > score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i % 1000 == 0:
            print('iter', i, ':', create_ciphertext(
                cipher_text, current_cipher)[0:99])
    return best_state

In [3]:
alphabet = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
cipher = 'мюное шхкьаёъыпивжлтзэбгйщчядрцуфс'

stop_symbols = string.punctuation + '\n\xa0«»\t—…'

In [4]:
duplets_frequency_in_corpus = compute_frequency_in_corpus('corpus.txt')

In [5]:
test_message = 'Англичане утверждают, что немецкий народ сопротивляется тотальным военным мерам правительства. Будто он хочет не тотальной войны, а капитуляции. Я спрашиваю вас: хотите ли вы тотальной войны? Хотите ли вы вести ее, даже если надо вести ее еще тотальнее и радикальнее, чем мы сегодня вообще можем себе представить? Фюрер ждет от нас таких свершений, которые затмят все, что было до сих пор. Мы хотим быть на высоте его требований. Мы гордимся им, а он должен иметь возможность гордиться нами <…> Нация готова ко всему. Фюрер приказал, мы следуем за ним. В этот час национального осмысления и внутреннего подъема мы еще вернее и нерушимее будем верить в победу. Мы видим победу в осязаемой близи, нам надо только протянуть руку и схватить ее. Мы должны найти в себе решимость поставить на службу ей абсолютно все. Таково веление времени. А потому наш лозунг: «Вставай, народ, да грядет буря!»'
test_text = clear_message(test_message, stop_symbols)
test_cipher = get_random_key()
test_ciphertext = create_ciphertext(test_text, test_cipher)

In [6]:
best_state = MCMC_decrypt(20000, test_ciphertext, duplets_frequency_in_corpus)
print('\n', 'MCMC decode:', best_state)
print('\n', 'True decode:', reverse_cipher(test_cipher))
print('\n', 'Decoded Text:', create_ciphertext(test_ciphertext, best_state))

iter 0 : тн яоютнпмхуьпрфэтзумюужмнпгпкбодмнтржэмйжиржуоьяцпуйцмужутяёнагмьжпннагмгпртгмиртьоупяёйуьтмчхэужм
iter 1000 : ингдуйинаеытлавзриютейтоенасахбучениворе оквотулдпат петотидьняселоаннясесависеквилутадь тлиемыртое
iter 2000 : инглуйина ытпавзриют йто насацшуч нивор еоквотуплдатед тотильняс поанняс савис квипутальетпи мырто 
iter 3000 : англичане ытдержвают что несецмий наров копротидлуетку тотальняс доенняс серас прадительктда бывто 
iter 4000 : англичане ытверждают что несефмий народ копротивлуетку тотальняс военняс серас правительктва быдто 
iter 5000 : англичане ытверждают что несефмий народ копротивлуетку тотальняс военняс серас правительктва быдто 
iter 6000 : англичане ытверждают что несефмий народ копротивлуетку тотальняс военняс серас правительктва быдто 
iter 7000 : англичане ытверждают что несешмий народ копротивлуетку тотальняс военняс серас правительктва быдто 
iter 8000 : англичане ытверждают что несецмий народ копротивлуетку тотальняс военняс серас правительктва бы

In [7]:
ciphertext = 'щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз'
best_state = MCMC_decrypt(20000, ciphertext, duplets_frequency_in_corpus)
print('\n', 'MCMC decode:', best_state)
print('\n', 'True decode:', cipher)
print('\n', 'Decoded Text:', create_ciphertext(ciphertext, best_state))

iter 0 : щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдс
iter 1000 : ю чуть не залыдар от жарости к сехе ведь мне тлидгать шесть рет восемнадгать из ниб ю лахотай что т
iter 2000 : я чуть не зарылам от бамости к сехе вель дне трилжать фесть мет воседналжать из ниг я рахотаю что т
iter 3000 : я чуть не зарылад от шадости к себе вель мне трилжать фесть дет восемналжать из них я работаю что т
iter 4000 : я чуть не зарылад от цадости к себе вель мне трилгать жесть дет восемналгать из них я работаю что т
iter 5000 : я чуть не зарылад от шадости к себе вель мне трилгать жесть дет восемналгать из них я работаю что т
iter 6000 : я чуть не зарылад от жадости к себе вель мне трилгать щесть дет восемналгать из них я работаю что т
iter 7000 : я чуть не зарыдал от шалости к сеге ведь мне тридцать жесть лет восемнадцать из них я раготаю что т
iter 8000 : я чуть не зарылад от жадости к себе вель мне трилгать фесть дет восемналгать из них я работаю ч

In [8]:
compute_score(ciphertext, cipher, duplets_frequency_in_corpus)

2806.686026773373

In [9]:
compute_score(ciphertext, best_state, duplets_frequency_in_corpus)

2807.888364416493